In [ ]:
# add path to sys to import functions
import os 
import sys
dir_path = os.getcwd()
sys.path.insert(0,os.path.abspath(os.path.join(dir_path, '..')))

# Mitigating Disparities

This demo shows how to run the `mitigate_disparity` scripot on a development dataset. 
Below, we demonstrate how to run `mitgate_disparity.py` from the command line using a model trained to predict risk of admission to the emergency department using the freely available [MIMIC-IV repository](https://www.nature.com/articles/s41597-022-01899-x). 

## Inputs

In addition to providing a dataset, the user should identify protected features by providing a list of column names corresponding to demographics and/or other variables over which fairness should be sought.

## Continuous Updating

This script may also be used to update a model with new data by passing a `starting_point` parameter. 
This allows models to be continously updated over time as new biases arise and dataset shift occurs, without having to start from scratch. 
Under the hood, this is done by setting `checkpoint=True` in the `FomoClassifier` object. 
See the [Fomo docs](https://cavalab.org/fomo/) for more information on options. 

You may also [browse the API](https://cavalab.org/interfair/api.html) for `mitigate_disparity.py`. 


Below, we run `mitigate_disparity.py` using a development dataset and specifying that we want to ensure fairness with respect to the features named ethnicity, gender, and insurance. 

In [ ]:
%run ../mitigate_disparity.py \
    --dataset ../data/mimic/development_dataset.train.csv \
    --protected_features ethnicity,gender,insurance \
    --starting_point "checkpoint.25e21a10-80f0-4c4c-8b26-83243e32bd2a.pkl"

In [ ]:
Calling `mitigate_disparity.py` will produce an `estimator.pkl` file that can be loaded for further analysis. 
We demonstrate this below

## Visualize fairness/error trade-offs

Once training is done, we can view a set of candidate models. 
The red dot indicates the model that was selected. 
In addition to the default "PseudoWeights" approach, FOMO provides other multi-criteria decsion making (MCDM) algorithms via pymoo.

In [ ]:
import pickle
with open('../estimator.pkl','rb') as f:
    est = pickle.load(f)
est.plot().show()

## save video of optimization

We can view progress of the algorithm by stepping through the optimization history. 
This is demonstrated below. 

In [ ]:
from pyrecorder.recorder import Recorder
from pyrecorder.writers.video import Video
# from pyrecorder.writers.streamer import Streamer
from pymoo.visualization.scatter import Scatter
import matplotlib.pyplot as plt

with open('../estimator.pkl','rb') as f:
    est = pickle.load(f)
# use the video writer as a resource
filename = "xgb_nsga3_mlp_85.mp4"

# from pyrecorder.writers.gif import GIF
# result = alg.result()
result = est.res_

with Recorder(Video(filename, fps=10)) as rec:
# with Recorder(GIF(filename, duration=10)) as rec:
    # for each algorithm object in the history
    for entry in result.history:
        sc = Scatter(title=("Gen %s" % entry.n_gen),
                     labels=['Overall False Positive Rate (FPR)', 'Subgroup False Negative Rate (FNR) Violation']
                    )
        sc.add(entry.pop.get("F"))
#         sc.add(entry.pop.get("F"), plot_type="line", color="black", alpha=0.7)
        sc.do()
        plt.xlim([0.045, 0.085])
        plt.ylim([0.04, 0.41])
        # finally record the current visualization to the video
        rec.record()
    

## check test set performance

In [ ]:
from utils import make_measure_dataset
import pandas as pd

import pickle
with open('../estimator.pkl','rb') as f:
    est = pickle.load(f)
    
df_test = pd.read_csv('../data/mimic/development_dataset.test.csv')
X_test = df_test.drop(columns='binary outcome')
y_test = df_test['binary outcome']
make_measure_dataset(est, 'fomo', X_test, y_test)

## measure change in disparity measures

Now that we have an updated model, we can check how our disparity measures have changed. 
Below we run `measure_disparity.py` with our new results and compare the results to the old ones. 

In [ ]:
from measure_disparity import measure_disparity
measure_disparity('../fomo_model_mimic4_admission.csv', savefile='df_fairness.post.csv')

## Improvements over Baseline Model

If we compare with results from our baseline model in [demo_measure_disparity.ipynb](https://cavalab.org/interfair/demo_measure_disparity.html), we see that we have made a marked improvement to the maximum subgroup deviations on the test set:


### Brier Score (MSE)

- Subgroup: ethnicity=WHITE
- Original: 19.9%, New: 19.3%

### FNR

- Original: 20.4%, New: 10.9% 

### FPR

- Subgroup: insurance=Medicare,ethnicity=AMERICAN INDIAN/ALASKA NATIVE,gender=F
- FPR is 86.0 % higher among this group than the population.
- Original: 86.0%, New: 62.3%

### Positivity Rate

- Original: 44.9 %, New: 28.8% 


In summary, our new model has a more equal false negative rate among groups than before, which was our goal. 
In addition, we see reductions in the false positive rate deviations and differences in positivity rates. 

In terms of overall performance, we see a slight decrease, as we would also expect:

- AUROC: 0.881 -> 0.859
- AUPRC: 0.77 -> 0.74


By using the model visualization tools above, decision makers can decide whether this model, or another within the set, is a better fit to the use case, as needed. 